In [ ]:
# 🧬 Unity Catalog Multi-Layer Lineage Demo
# Catalog: `unity_demo`, Schema: `governance_lab`
# This notebook creates 3 base tables and 3 derived tables/views, each linked by joins or aggregations.
# The Unity Catalog Lineage UI will reflect these layers.

spark.sql("USE CATALOG unity_demo")
spark.sql("USE SCHEMA governance_lab")


In [ ]:
# Table 1: lineage_customers – customer reference data
from pyspark.sql import Row
customers = [
    Row(customer_id=1, name="Alice", region="UK"),
    Row(customer_id=2, name="Bob", region="EU"),
    Row(customer_id=3, name="Charlie", region="US")
]
spark.createDataFrame(customers).write.mode("overwrite").saveAsTable("unity_demo.governance_lab.lineage_customers")


In [ ]:
# Table 2: lineage_products – product reference data
products = [
    Row(product_id=101, name="Laptop", category="Electronics"),
    Row(product_id=102, name="Headphones", category="Electronics"),
    Row(product_id=103, name="Mug", category="Homeware")
]
spark.createDataFrame(products).write.mode("overwrite").saveAsTable("unity_demo.governance_lab.lineage_products")


In [ ]:
# Table 3: lineage_orders – high-level orders
orders = [
    Row(order_id=1001, customer_id=1, order_date="2024-01-01"),
    Row(order_id=1002, customer_id=1, order_date="2024-01-02"),
    Row(order_id=1003, customer_id=2, order_date="2024-01-03")
]
spark.createDataFrame(orders).write.mode("overwrite").saveAsTable("unity_demo.governance_lab.lineage_orders")


In [ ]:
# Layer 1: lineage_order_details – exploded orders with products and quantities
spark.sql("""
CREATE OR REPLACE TABLE unity_demo.governance_lab.lineage_order_details AS
SELECT o.order_id, o.customer_id, o.order_date, p.product_id, p.name AS product_name, p.category, 
       CASE o.order_id 
            WHEN 1001 THEN 2
            WHEN 1002 THEN 1
            ELSE 3
        END AS quantity,
       CASE p.product_id
            WHEN 101 THEN 1000.0
            WHEN 102 THEN 200.0
            ELSE 10.0
       END AS unit_price
FROM unity_demo.governance_lab.lineage_orders o
JOIN unity_demo.governance_lab.lineage_products p
    ON p.product_id IN (101, 102, 103)
""")


In [ ]:
# Layer 2: lineage_revenue_by_customer – revenue totals per customer
spark.sql("""
CREATE OR REPLACE TABLE unity_demo.governance_lab.lineage_revenue_by_customer AS
SELECT customer_id,
       SUM(quantity * unit_price) AS total_spend
FROM unity_demo.governance_lab.lineage_order_details
GROUP BY customer_id
""")


In [ ]:
# Layer 3: lineage_top_customers – joined with customer names, top spenders
spark.sql("""
CREATE OR REPLACE TABLE unity_demo.governance_lab.lineage_top_customers AS
SELECT c.customer_id, c.name, r.total_spend
FROM unity_demo.governance_lab.lineage_revenue_by_customer r
JOIN unity_demo.governance_lab.lineage_customers c
  ON r.customer_id = c.customer_id
WHERE r.total_spend > 500
""")


In [ ]:
# ✅ Lineage Demo Complete
# Visit any table in the Unity Catalog Lineage tab, especially `lineage_top_customers`,
# and observe dependencies all the way back to:
# - `lineage_customers`
# - `lineage_orders`
# - `lineage_products`
# Docs: https://docs.databricks.com/en/data-governance/unity-catalog/lineage.html
